In [184]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/cifar10-python/cifar-10-python.tar.gz
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_1
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_2
/kaggle/input/cifar10-python/cifar-10-batches-py/batches.meta
/kaggle/input/cifar10-python/cifar-10-batches-py/test_batch
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_3
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_5
/kaggle/input/cifar10-python/cifar-10-batches-py/data_batch_4
/kaggle/input/cifar10-python/cifar-10-batches-py/readme.html


In [185]:
import keras
from keras.datasets import cifar10
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)


In [ ]:
print("Input image shape:", x_train.shape[1:])

In [186]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train.reshape(-1, 32 * 32 * 3)).reshape(-1, 32, 32, 3)
x_val = scaler.transform(x_val.reshape(-1, 32 * 32 * 3)).reshape(-1, 32, 32, 3)
x_test = scaler.transform(x_test.reshape(-1, 32 * 32 * 3)).reshape(-1, 32, 32, 3)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
def create_model():
    m1=3
    m2=3
    x1=64
    x2=32
    d=0.2
    x3=4096

    model = Sequential()

    model.add(Conv2D(filters=x1, kernel_size=(m1, m1), activation='relu', input_shape=(32, 32, 3),padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=x2, kernel_size=(m2, m2), activation='relu',padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu',padding="same"))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))

    model.add(Flatten())

    model.add(Dense(x3, activation='relu'))
    model.add(Dropout(0.2))

    model.add(Dense(10, activation='softmax'))
    return model



In [ ]:
import matplotlib.pyplot as plt
from keras.callbacks import ModelCheckpoint
learning_rate = 0.001
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
model=create_model();
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
epochs = 20
checkpoint = ModelCheckpoint("best_model.h5",  monitor='val_loss', save_best_only=True, mode='min' )

In [ ]:
history = model.fit(x_train, y_train, epochs=epochs,validation_data=(x_val, y_val),callbacks=[checkpoint])

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
learning_rates = [0.0001, 0.001, 0.01, 0.1]
epochs = 20
history_dict = {}

for lr in learning_rates:
    model=create_model()
    learning_rate =lr
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    checkpoint = ModelCheckpoint(f"best_model_lr_{lr}.h5", monitor='val_loss', save_best_only=True, mode='min')
    history = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val),callbacks=[checkpoint])
    history_dict[lr] = history



In [ ]:
for i, (lr, history) in enumerate(history_dict.items(), 1):
    plt.figure(figsize=(8, 6))

    plt.plot(history.history['loss'], label=f'Training Loss (LR={lr})')
    plt.plot(history.history['val_loss'], label=f'Validation Loss (LR={lr})')

    plt.title(f'Training and Validation Loss for LR={lr}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [ ]:
print(y_train[1])

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score


In [ ]:
model_path = "/kaggle/working/best_model_lr_0.001.h5"
model = load_model(model_path)

In [ ]:
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)


In [ ]:
# Calculate accuracy
test_accuracy = accuracy_score(y_test, y_pred_classes)
print(f'Test Accuracy: {test_accuracy}')

# confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print('Confusion Matrix:')
print(conf_matrix)

# precision and recall
precision = precision_score(y_test, y_pred_classes, average='weighted')
recall = recall_score(y_test, y_pred_classes, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')


In [191]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.applications import ResNet50
from keras.models import Sequential

# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Create a fine-tuned model
def transfer_learning():
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.15))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    return model

In [ ]:
learning_rates = [0.0015]
epochs = 20
history1_dict = {}

for lr in learning_rates:
    model=transfer_learning()
    learning_rate =lr
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    checkpoint = ModelCheckpoint(f"best_model_lr_transfer{lr}.h5", monitor='val_loss', save_best_only=True, mode='min')
    history1 = model.fit(x_train, y_train, epochs=epochs, validation_data=(x_val, y_val),callbacks=[checkpoint], batch_size=200)
    history1_dict[lr] = history1

Epoch 1/20
200/200 [==============================] - 10s 33ms/step - loss: 2.0883 - accuracy: 0.2615 - val_loss: 1.8174 - val_accuracy: 0.3396
Epoch 2/20
200/200 [==============================] - 5s 27ms/step - loss: 1.7273 - accuracy: 0.3775 - val_loss: 1.6592 - val_accuracy: 0.3981
Epoch 3/20
200/200 [==============================] - 5s 26ms/step - loss: 1.6398 - accuracy: 0.4140 - val_loss: 1.6127 - val_accuracy: 0.4226
Epoch 4/20
200/200 [==============================] - 5s 27ms/step - loss: 1.5945 - accuracy: 0.4294 - val_loss: 1.5596 - val_accuracy: 0.4420
Epoch 5/20
200/200 [==============================] - 5s 23ms/step - loss: 1.5665 - accuracy: 0.4403 - val_loss: 1.5880 - val_accuracy: 0.4369
Epoch 6/20
200/200 [==============================] - 5s 27ms/step - loss: 1.5379 - accuracy: 0.4540 - val_loss: 1.5416 - val_accuracy: 0.4496
Epoch 7/20
200/200 [==============================] - 6s 28ms/step - loss: 1.5131 - accuracy: 0.4595 - val_loss: 1.5402 - val_accuracy: 0.458

In [ ]:
for i, (lr, history1) in enumerate(history1_dict.items(), 1):
    plt.figure(figsize=(8, 6))

    plt.plot(history1.history['loss'], label=f'Training Loss (LR={lr})')
    plt.plot(history1.history['val_loss'], label=f'Validation Loss (LR={lr})')

    plt.title(f'Training and Validation Loss for LR={lr}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()